<a href="https://colab.research.google.com/github/jorcisai/ARF/blob/master/HuggingFace/06-Finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning

Motivation:
* Adaptation of pre-trained Transformer models to a specific task
* Leveraging models trained on massive datasets
* Saving time and power

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

Taking a look at the content of the Europarl-ST dataset having English as a source language

In [44]:
from datasets import load_dataset

raw_datasets = load_dataset("tj-solergibert/Europarl-ST-processed-mt-en")
print(raw_datasets)

  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    test: Dataset({
        features: ['source_text', 'dest_text', 'dest_lang'],
        num_rows: 86170
    })
    train: Dataset({
        features: ['source_text', 'dest_text', 'dest_lang'],
        num_rows: 602605
    })
    valid: Dataset({
        features: ['source_text', 'dest_text', 'dest_lang'],
        num_rows: 81968
    })
})


The original dataset includes the translation from English into several European languages

In [9]:
raw_datasets["train"].features

{'source_text': Value(dtype='string', id=None),
 'dest_text': Value(dtype='string', id=None),
 'dest_lang': ClassLabel(names=['de', 'en', 'es', 'fr', 'it', 'nl', 'pl', 'pt', 'ro'], id=None)}

Let us take a look at the translations of the first English sentence

In [15]:
raw_datasets["train"][:7]["source_text"]

['Since 1977, most financial services, including insurance and investment fund management, have been exempt from VAT.',
 'Since 1977, most financial services, including insurance and investment fund management, have been exempt from VAT.',
 'Since 1977, most financial services, including insurance and investment fund management, have been exempt from VAT.',
 'Since 1977, most financial services, including insurance and investment fund management, have been exempt from VAT.',
 'Since 1977, most financial services, including insurance and investment fund management, have been exempt from VAT.',
 'Since 1977, most financial services, including insurance and investment fund management, have been exempt from VAT.',
 'Since 1977, most financial services, including insurance and investment fund management, have been exempt from VAT.']

In [14]:
raw_datasets["train"][:7]["dest_text"]

['Seit 1977 wurden die meisten Finanzdienstleistungen, einschließlich Versicherungen und Verwaltung von Investmentfunds, von der Mehrwertsteuer ausgenommen.',
 'La mayoría de los servicios financieros, incluidos los seguros y la gestión de fondos de inversión, están exentos de IVA desde 1977.',
 'Depuis 1977, la plupart des services financiers, dont les assurances et la gestion des fonds de placement, ne sont pas tenus d ’ appliquer une TVA.',
 'Dal 1997 la maggior parte dei servizi finanziari, compresi i servizi assicurativi e la gestione di fondi di investimento, sono esenti da IVA.',
 'Sinds 1977 zijn de meeste financiële diensten, met inbegrip van verzekeringen en het beheer van beleggingsfondsen, vrijgesteld van btw.',
 'większość usług finansowych, w tym usług w zakresie ubezpieczeń i zarządzania funduszami inwestycyjnymi, była zwolniona z opodatkowania podatkiem VAT.',
 'Desde 1977 que a maioria dos serviços financeiros, incluindo os seguros e a gestão de fundos de investimento,

In [16]:
raw_datasets["train"][:7]["dest_lang"]

[0, 2, 3, 4, 5, 6, 7]

Filtering Europarl-ST only for English into Spanish using a simple [lambda function](https://realpython.com/python-lambda/) with the [Dataset.filter() function](https://huggingface.co/docs/datasets/v2.9.0/en/package_reference/main_classes#datasets.Dataset.filter)

In [45]:
lang="es"
lang_id = raw_datasets["train"].features["dest_lang"].names.index(lang)
raw_datasets["train"] = raw_datasets["train"].filter(lambda x: x["dest_lang"] == lang_id)

A good practice is to take a small random sample to get a quick feel for the type of data you’re working with. In Datasets, we can create a random sample by chaining the [Dataset.shuffle()](https://huggingface.co/docs/datasets/v2.9.0/en/package_reference/main_classes#datasets.Dataset.shuffle) and [Dataset.select()](https://huggingface.co/docs/datasets/v2.9.0/en/package_reference/main_classes#datasets.Dataset.select) functions together:

In [18]:
raw_sample = raw_datasets["train"].shuffle(seed=13).select(range(5))
raw_sample[:5]

{'source_text': ['the general issues of religious freedom, this is an absolute scandal and I welcome the fact that a number of Christian bishops from Iraq are to travel to Strasbourg in December',
  'The Commission fully supports the work of the International Criminal Tribunal for Yugoslavia, the ICTY.',
  'Another boost will come following the wonderful achievement of Swansea City Football Club – and I say this for the benefit of my colleague Richard Howitt – who have just deservedly reached the Premier Football League after a fantastic season.',
  'In conclusion, I would like to thank Parliament for its support and constructive approach.',
  'The European Council endorses the long-term vision for biodiversity by 2050 and the 2020 target set forth in the aforementioned Council conclusions of 15 March 2010.'],
 'dest_text': ['de los problemas de libertad religiosa, esto es un escándalo absoluto y celebro que varios obispos cristianos iraquíes vayan a viajar a Estrasburgo en diciembre',

Now we load the pre-trained tokenizer for the mT5 model and apply it to a sample English-Spanish pair

In [46]:
from transformers import AutoTokenizer

checkpoint = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenized_sample = tokenizer(raw_datasets["train"][0]["source_text"],raw_datasets["train"][0]["dest_text"])

/usr/local/lib/python3.8/dist-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [40]:
print(tokenizer.convert_ids_to_tokens(tokenized_sample["input_ids"]))

['▁', 'Since', '▁1977', ',', '▁most', '▁', 'financial', '▁services', ',', '▁', 'including', '▁insurance', '▁and', '▁investment', '▁fund', '▁management', ',', '▁have', '▁been', '▁exemp', 't', '▁from', '▁VAT', '.', '</s>', '▁La', '▁mayor', 'ía', '▁de', '▁los', '▁', 'servicios', '▁financier', 'os', ',', '▁inclui', 'dos', '▁los', '▁', 'seguro', 's', '▁', 'y', '▁la', '▁', 'g', 'estión', '▁de', '▁fond', 'os', '▁de', '▁in', 'versión', ',', '▁está', 'n', '▁ex', 'ent', 'os', '▁de', '▁', 'IVA', '▁', 'desde', '▁1977', '.', '</s>']


In [39]:
tokenizer.decode(tokenized_sample["input_ids"])

'Since 1977, most financial services, including insurance and investment fund management, have been exempt from VAT.</s> La mayoría de los servicios financieros, incluidos los seguros y la gestión de fondos de inversión, están exentos de IVA desde 1977.</s>'

We can apply the tokenizer function to any dataset

In [ ]:
tokenized_dataset = tokenizer(raw_datasets["test"]["source_text"],raw_datasets["test"]["dest_text"],padding=True)
print(tokenized_dataset)

In [26]:
def tokenize_function(sample):
    return tokenizer(sample["source_text"], sample["dest_text"])

In [27]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

  0%|          | 0/87 [00:00<?, ?ba/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  0%|          | 0/77 [00:00<?, ?ba/s]

  0%|          | 0/82 [00:00<?, ?ba/s]

DatasetDict({
    test: Dataset({
        features: ['source_text', 'dest_text', 'dest_lang', 'input_ids', 'attention_mask'],
        num_rows: 86170
    })
    train: Dataset({
        features: ['source_text', 'dest_text', 'dest_lang', 'input_ids', 'attention_mask'],
        num_rows: 76469
    })
    valid: Dataset({
        features: ['source_text', 'dest_text', 'dest_lang', 'input_ids', 'attention_mask'],
        num_rows: 81968
    })
})

In [28]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [29]:
samples = tokenized_datasets["train"][:8]
samples = {k: v for k, v in samples.items() if k not in ["source_text", "dest_text", "dest_lang"]}
[len(x) for x in samples["input_ids"]]

[67, 122, 98, 93, 86, 106, 84, 167]

In [30]:
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': torch.Size([8, 167]), 'attention_mask': torch.Size([8, 167])}